Requests
========
*Chuan-Zheng Lee*

This notebook demonstrates the `Requests.jl` package. It was created using Julia version 0.5.0-dev+539 (commit 92670a9). 

`Requests` is a basic HTTP client. It can be used to download web pages, or (perhaps more usefully) interact with HTTP-based APIs. For example, some web services might provide data sets over HTTP.

This notebook only demonstrates the `get()` function, since it's the most useful. There's also `post()`,
which might occasionally be used, and `put()`, `delete()` and `options()`, which probably almost never
get used.

More information about this package can be found at https://github.com/JuliaWeb/Requests.jl.

To install:
```
Pkg.add("Requests")
```

In [1]:
using Requests
using Interact

Status code
------------

The function `get(url)`, which returns a `Response` object, which encapsulates an HTTP response.

You can read the HTTP status code with `statuscode()`. (The status code 200 means "OK", 404 means "Not Found", etc.)

In [2]:
# Request web pages and report the status code of the response
@show julia_homepage = get("http://julialang.org")
@show statuscode(julia_homepage) # should give 200 (OK)

@show nonexistent_page = get("http://web.stanford.edu/~nonexistent")
@show statuscode(nonexistent_page) # should give 404 (Not Found)
;

julia_homepage = get("http://julialang.org") = Response(200 OK, 22 headers, 28858 bytes in body)
statuscode(julia_homepage) = 200
nonexistent_page = get("http://web.stanford.edu/~nonexistent") = Response(404 Not Found, 11 headers, 5847 bytes in body)
statuscode(nonexistent_page) = 404


Headers
-------

The function `headers()` returns a `Dict` of all the headers in the response.

In [3]:
# Retrieve a web page and display the headers in the response
@show facebook_homepage = get("https://www.facebook.com/")
headers(facebook_homepage)

facebook_homepage = get("https://www.facebook.com/") = Response(200 OK, 20 headers, 52751 bytes in body)

Dict{AbstractString,AbstractString} with 20 entries:
  "Connection"             => "keep-alive"
  "Date"                   => "Fri, 09 Oct 2015 04:44:27 GMT"
  "Public-Key-Pins-Report… => "max-age=500; pin-sha256=\"WoiWRyIOVNa9ihaBciRSC7…
  "http_minor"             => "1"
  "Keep-Alive"             => "1"
  "status_code"            => "200"
  "Transfer-Encoding"      => "chunked"
  "Cache-Control"          => "private, no-cache, no-store, must-revalidate"
  "X-FB-Debug"             => "K3dLGYH71YOiCi8A939JIVgcwy/K3pUUqBDPUKXUaexKKLcL…
  "Server"                 => "Julia/0.5.0-dev+539"
  "Expires"                => "Sat, 01 Jan 2000 00:00:00 GMT"
  "X-XSS-Protection"       => "0"
  "X-Content-Type-Options" => "nosniff"
  "Vary"                   => "Accept-Encoding"
  "Pragma"                 => "no-cache"
  "http_major"             => "1"
  "Strict-Transport-Secur… => "max-age=15552000; preload"
  "Content-Type"           => "text/html"
  "X-Frame-Options"        => "DENY"
  "Content-

Interactive example
-------------------

This example accesses the results of New Zealand general elections, which are hosted in CSV format by electorate and year on the New Zealand Electoral Commission's website. It then parses the CSV file to extract information about how many votes each party received in that electorate in that year, and displays the result as a simple HTML table.

In real life one probably wouldn't do something so dull as display the information in an HTML table. One could, for example, use this method to analyze how a party's popularity changes over time, or to conduct other more involved analyses.

In [4]:
# Download the results of some New Zealand general election in some electorate,
# identified by electorate ID. Display the party votes as a simple HTML table.

@manipulate for electorate_id in 1:71, year in 2002:3:2014
    
    # Construct URL to download
    url = @sprintf("http://electionresults.govt.nz/electionresults_%d/e9/csv/e9_part8_%s_%d.csv", year, "party", electorate_id)
    
    response = get(url) # retrieve the response
    @assert(statuscode(response) == 200)
    
    content = readbytes(response) # reads the contents as a byte stream
    data = readcsv(content)
    
    # Everything from here onwards is processing the downloaded data, generating
    # a simple HTML display of the votes for each party in this electorate.
    text = [UTF8String("""<p><b>Party votes in the $year New Zealand general election</b></p>
        <p><i>For electorate: <b>$(data[2,1])</b></i></p>
        <table>
        <tr>
            <th>Party</th>
            <th style="text-align: right;">Votes</th>
            <th style="text-align: right;">%age</th>
        </tr>
        """)]

    # The CSV file contains counts for each polling booth, and then the totals at the end.
    # We only care about the totals. Also, sort in descending order by number of votes.
    first_party_row = find(x -> isa(x, AbstractString) && contains(x, "Electorate Party Valid Votes"), data)[1] + 1
    party_vote_data = sortrows(data[first_party_row:size(data,1), 1:3], by=x->x[2], rev=true)
    for row in 1:size(party_vote_data,1)
        party_name = party_vote_data[row,1]
        votes = party_vote_data[row,2]
        percentage = @sprintf("%.2f%%", party_vote_data[row,3] * 100)
        push!(text, """<tr>
                <td> $party_name </td>
                <td style="text-align: right; width: 5em;"> $votes </td>
                <td style="text-align: right; width: 5em;"> $percentage </td>
            </tr>
            """)
    end
    
    push!(text, "</table>")
    html(string(text...))
end

Interact.Slider{Int64}([Reactive.Input{Int64}] 36,"electorate_id",36,1:71)

Interact.Slider{Int64}([Reactive.Input{Int64}] 2008,"year",2008,2002:3:2014)

Interact.HTML("","<p><b>Party votes in the 2008 New Zealand general election</b></p>\n<p><i>For electorate: <b>Otaki 36</b></i></p>\n<table>\n<tr>\n    <th>Party</th>\n    <th style=\"text-align: right;\">Votes</th>\n    <th style=\"text-align: right;\">%age</th>\n</tr>\n<tr>\n    <td> National Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 17534 </td>\n    <td style=\"text-align: right; width: 5em;\"> 44.77% </td>\n</tr>\n<tr>\n    <td> Labour Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 14472 </td>\n    <td style=\"text-align: right; width: 5em;\"> 36.95% </td>\n</tr>\n<tr>\n    <td> Green Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 2207 </td>\n    <td style=\"text-align: right; width: 5em;\"> 5.64% </td>\n</tr>\n<tr>\n    <td> New Zealand First Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 2153 </td>\n    <td style=\"text-align: right; width: 5em;\"> 5.50% </td>\n</tr>\n<tr>\n    <td> ACT New Zealand </td>\n    <td style=\"text-align: right; width: 5em;\"> 1014 </td>\n    <td style=\"text-align: right; width: 5em;\"> 2.59% </td>\n</tr>\n<tr>\n    <td> Jim Anderton's Progressive </td>\n    <td style=\"text-align: right; width: 5em;\"> 453 </td>\n    <td style=\"text-align: right; width: 5em;\"> 1.16% </td>\n</tr>\n<tr>\n    <td> United Future </td>\n    <td style=\"text-align: right; width: 5em;\"> 397 </td>\n    <td style=\"text-align: right; width: 5em;\"> 1.01% </td>\n</tr>\n<tr>\n    <td> Maori Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 321 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.82% </td>\n</tr>\n<tr>\n    <td> The Bill and Ben Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 177 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.45% </td>\n</tr>\n<tr>\n    <td> Kiwi Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 165 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.42% </td>\n</tr>\n<tr>\n    <td> Aotearoa Legalise Cannabis Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 115 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.29% </td>\n</tr>\n<tr>\n    <td> Family Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 84 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.21% </td>\n</tr>\n<tr>\n    <td> Alliance </td>\n    <td style=\"text-align: right; width: 5em;\"> 28 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.07% </td>\n</tr>\n<tr>\n    <td> Libertarianz </td>\n    <td style=\"text-align: right; width: 5em;\"> 12 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.03% </td>\n</tr>\n<tr>\n    <td> Democrats for Social Credit </td>\n    <td style=\"text-align: right; width: 5em;\"> 10 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.03% </td>\n</tr>\n<tr>\n    <td> New Zealand Pacific Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 9 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.02% </td>\n</tr>\n<tr>\n    <td> Workers Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 6 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.02% </td>\n</tr>\n<tr>\n    <td> The Republic of New Zealand Party </td>\n    <td style=\"text-align: right; width: 5em;\"> 5 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.01% </td>\n</tr>\n<tr>\n    <td> RAM - Residents Action Movement </td>\n    <td style=\"text-align: right; width: 5em;\"> 0 </td>\n    <td style=\"text-align: right; width: 5em;\"> 0.00% </td>\n</tr>\n</table>")